In [1]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from numpy import concatenate
from math import sqrt
import random
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/Users/alexellard/Downloads/scenarios_data.csv', header=0, index_col=0)
df

,reservoir_demand,node_1_demand,node_2_demand,node_3_demand,node_4_demand,node_5_demand,node_6_demand,node_7_demand,node_8_demand,node_9_demand,...,link_23_flow,link_24_flow,link_25_flow,link_26_flow,link_27_flow,link_28_flow,link_29_flow,link_30_flow,link_31_flow,link_32_flow
Timestamp,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,-3348.0,154.8,169.2,18.0,118.8,169.2,234.0,86.4,82.8,104.4,...,439.2,-205.2,-75.6,-18.0,118.8,68.4,7.2,-54.0,75.6,212.4
2017-01-01 00:30:00,-2959.2,136.8,147.6,18.0,108.0,144.0,237.6,79.2,75.6,90.0,...,392.4,-187.2,-72.0,-21.6,108.0,61.2,3.6,-54.0,72.0,183.6
2017-01-01 01:00:00,-2692.8,129.6,126.0,14.4,104.4,133.2,205.2,72.0,72.0,90.0,...,349.2,-176.4,-64.8,-18.0,93.6,50.4,3.6,-39.6,54.0,151.2
2017-01-01 01:30:00,-2379.6,104.4,100.8,10.8,86.4,122.4,180.0,68.4,57.6,82.8,...,316.8,-158.4,-54.0,-10.8,79.2,46.8,7.2,-32.4,50.4,140.4
2017-01-01 02:00:00,-2228.4,104.4,104.4,10.8,79.2,122.4,183.6,68.4,54.0,75.6,...,277.2,-144.0,-57.6,-18.0,72.0,39.6,7.2,-32.4,43.2,118.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-31 21:30:00,-5306.4,205.2,208.8,39.6,183.6,284.4,316.8,176.4,133.2,129.6,...,604.8,-259.2,32.4,115.2,169.2,100.8,10.8,-100.8,122.4,309.6
2017-12-31 22:00:00,-4910.4,208.8,183.6,39.6,154.8,252.0,295.2,165.6,133.2,133.2,...,565.2,-241.2,21.6,100.8,154.8,86.4,18.0,-75.6,100.8,288.0
2017-12-31 22:30:00,-4683.6,187.2,183.6,36.0,147.6,230.4,320.4,147.6,118.8,126.0,...,529.2,-226.8,21.6,100.8,147.6,86.4,14.4,-86.4,108.0,266.4


In [3]:
# convert series to supervised learning
def series_to_supervised(data, n_in, n_out, vars_to_predict=None):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i-1))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(n_out):
        cols.append(df[vars_to_predict].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in vars_to_predict]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in vars_to_predict]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    return agg

values = df.values
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

# node 1 indexes
target_variable_demand_node1 = [1]
target_variable_pressures_node1 = [33]
target_variable_flows_node1 = [64]
#node 16 indexes
target_variable_demand_node16 = [16]
target_variable_pressures_node16 = [48]
target_variable_flows_node16 = [79]
#node 29 indexes
target_variable_demand_node29 = [29]
target_variable_pressures_node29 = [61]
target_variable_flows_node29 = [92]


reframed_demand_node1 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_demand_node1)
reframed_pressures_node1 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_pressures_node1)
reframed_flows_node1 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_flows_node1)

reframed_demand_node16 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_demand_node16)
reframed_pressures_node16 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_pressures_node16)
reframed_flows_node16 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_flows_node16)

reframed_demand_node29 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_demand_node29)
reframed_pressures_node29 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_pressures_node29)
reframed_flows_node29 = series_to_supervised(values, 1, 1, vars_to_predict=target_variable_flows_node29)

reframed_demand_node1 = reframed_demand_node1.drop(reframed_demand_node1.columns[1], axis=1)
reframed_pressures_node1 = reframed_pressures_node1.drop(reframed_pressures_node1.columns[33], axis=1)
reframed_flows_node1 = reframed_flows_node1.drop(reframed_flows_node1.columns[64], axis=1)

reframed_demand_node16 = reframed_demand_node16.drop(reframed_demand_node16.columns[16], axis=1)
reframed_pressures_node16 = reframed_pressures_node16.drop(reframed_pressures_node16.columns[48], axis=1)
reframed_flows_node16 = reframed_flows_node16.drop(reframed_flows_node16.columns[79], axis=1)

reframed_demand_node29 = reframed_demand_node29.drop(reframed_demand_node29.columns[29], axis=1)
reframed_pressures_node29 = reframed_pressures_node29.drop(reframed_pressures_node29.columns[61], axis=1)
reframed_flows_node29 = reframed_flows_node29.drop(reframed_flows_node29.columns[92], axis=1)

In [4]:
demands = [reframed_demand_node1, reframed_demand_node16, reframed_demand_node29]
pressures = [reframed_pressures_node1, reframed_pressures_node16, reframed_pressures_node29]
flows = [reframed_flows_node1, reframed_flows_node16, reframed_flows_node29]

parameters = [demands, pressures, flows]

In [ ]:
MAE_scaled = []
RMSE_scaled = []
R2_scaled = []
MAE = []
RMSE = []
R2 = []
sensors_covered = []

for run in range(1,11):
    for parameter in parameters:
        for node in parameter:
            #select random columns to drop
            columns_to_drop = random.sample(list(node.columns), 8)
            # Drop the selected columns from the dataframe
            if node.columns[-1] in columns_to_drop:
                columns_to_drop.remove(node.columns[-1])
            node.drop(columns_to_drop, axis=1, inplace=True)
            print('Node shape: ', node.shape)

            # split into train and test sets
            values = node.values
            scaled = scaler.fit_transform(values)
            train, test = train_test_split(scaled, test_size=0.2, random_state=0)

            # split into input and outputs
            train_X, train_y = train[:, :-1], train[:, -1]
            test_X, test_y = test[:, :-1], test[:, -1]

            # define the model
            model = RandomForestRegressor(n_estimators=50, random_state=0)

            # fit the model on the training data
            model.fit(train_X, train_y)

            # make predictions on the test data
            y_pred = model.predict(test_X)

            # calculate the mean squared error
            mae = mean_absolute_error(test_y, y_pred)
            MAE_scaled.append(mae)
            print(f'Run {run} Scaled Mean Absolute Error:', mae)
            rmse = sqrt(mean_squared_error(test_y, y_pred))
            RMSE_scaled.append(rmse)
            print(f'Run {run} Scaled RMSE: %.3f' % rmse)
            r2 = r2_score(test_y, y_pred)
            print(f'Run {run} Scaled R^2: %.3f' % r2)
            R2_scaled.append(r2)

            percentage_list = round(node.shape[1] / 96 * 100)
            sensors_covered.append(percentage_list)
            
            test_y = test_y.reshape((len(test_y), 1))
            inv_test_y = concatenate((test_y, test_X[:, -96:]), axis=1)
            test_y_inverse = scaler.inverse_transform(inv_test_y)
            test_y_inverse = test_y_inverse[:,0]

            y_pred = y_pred.reshape((len(y_pred), 1))
            inv_y_pred = concatenate((y_pred, test_X[:, -96:]), axis=1)
            y_pred_inverse = scaler.inverse_transform(inv_y_pred)
            y_pred_inverse = y_pred_inverse[:,0]

            # evaluate scaled errors
            mae_inverse = mean_absolute_error(test_y_inverse, y_pred_inverse)
            MAE.append(mae_inverse)
            print('Mean Absolute Error:', mae_inverse)
            rmse_inverse = sqrt(mean_squared_error(test_y_inverse, y_pred_inverse))
            RMSE.append(rmse_inverse)
            print('RMSE: %.3f' % rmse_inverse)
            r2_inverse = r2_score(test_y_inverse, y_pred_inverse)
            R2.append(r2_inverse)
            print('R^2: %.3f' % r2_inverse)

Node shape:  (87600, 88)
Run 1 Scaled Mean Absolute Error: 0.016821640999826123
Run 1 Scaled RMSE: 0.022
Run 1 Scaled R^2: 0.981
Mean Absolute Error: 143.5222452092041
RMSE: 0.022
R^2: 0.981
Node shape:  (87600, 89)
Run 1 Scaled Mean Absolute Error: 0.003813785433990362
Run 1 Scaled RMSE: 0.006
Run 1 Scaled R^2: 0.999
Mean Absolute Error: 32.53922367231296
RMSE: 0.006
R^2: 0.999
Node shape:  (87600, 88)
Run 1 Scaled Mean Absolute Error: 0.00888356208208455
Run 1 Scaled RMSE: 0.012
Run 1 Scaled R^2: 0.995
Mean Absolute Error: 75.79455466827729
RMSE: 0.012
R^2: 0.995
Node shape:  (87600, 88)
Run 1 Scaled Mean Absolute Error: 0.00021047891329412885
Run 1 Scaled RMSE: 0.001
Run 1 Scaled R^2: 1.000
Mean Absolute Error: 1.7958121343127267
RMSE: 0.001
R^2: 1.000
Node shape:  (87600, 88)
Run 1 Scaled Mean Absolute Error: 0.0005142637046232617
Run 1 Scaled RMSE: 0.001
Run 1 Scaled R^2: 1.000
Mean Absolute Error: 4.38769895446452
RMSE: 0.001
R^2: 1.000
Node shape:  (87600, 88)
Run 1 Scaled Mean 

Run 5 Scaled Mean Absolute Error: 0.0026990765615549257
Run 5 Scaled RMSE: 0.004
Run 5 Scaled R^2: 1.000
Mean Absolute Error: 23.02852289392319
RMSE: 0.004
R^2: 1.000
Node shape:  (87600, 56)
Run 5 Scaled Mean Absolute Error: 0.018225000380108894
Run 5 Scaled RMSE: 0.024
Run 5 Scaled R^2: 0.980
Mean Absolute Error: 9.51345093727853
RMSE: 0.024
R^2: 0.980
Node shape:  (87600, 49)
Run 6 Scaled Mean Absolute Error: 0.014814201344382954
Run 6 Scaled RMSE: 0.020
Run 6 Scaled R^2: 0.985
Mean Absolute Error: 126.39476860773965
RMSE: 0.020
R^2: 0.985
Node shape:  (87600, 50)


In [5]:
demand_errors_mae_node1 = MAE[::9]
demand_errors_mae_node16 = MAE[1::9]
demand_errors_mae_node29 = MAE[2::9]
pressure_errors_mae_node1 = MAE[3::9]
pressure_errors_mae_node16 = MAE[4::9]
pressure_errors_mae_node29 = MAE[5::9]
flow_errors_mae_node1 = MAE[6::9]
flow_errors_mae_node16 = MAE[7::9]
flow_errors_mae_node29 = MAE[8::9]

demand_errors_rmse_node1 = RMSE[::9]
demand_errors_rmse_node16 = RMSE[1::9]
demand_errors_rmse_node29 = RMSE[2::9]
pressure_errors_rmse_node1 = RMSE[3::9]
pressure_errors_rmse_node16 = RMSE[4::9]
pressure_errors_rmse_node29 = RMSE[5::9]
flow_errors_rmse_node1 = RMSE[6::9]
flow_errors_rmse_node16 = RMSE[7::9]
flow_errors_rmse_node29 = RMSE[8::9]

demand_errors_r2_node1 = R2[::9]
demand_errors_r2_node16 = R2[1::9]
demand_errors_r2_node29 = R2[2::9]
pressure_errors_r2_node1 = R2[3::9]
pressure_errors_r2_node16 = R2[4::9]
pressure_errors_r2_node29 = R2[5::9]
flow_errors_r2_node1 = R2[6::9]
flow_errors_r2_node16 = R2[7::9]
flow_errors_r2_node29 = R2[8::9]

demand_sensors_covered_node1 = sensors_covered[::9]
demand_sensors_covered_node16 = sensors_covered[1::9]
demand_sensors_covered_node29 = sensors_covered[2::9]
pressure_sensors_covered_node1 = sensors_covered[3::9]
pressure_sensors_covered_node16 = sensors_covered[4::9]
pressure_sensors_covered_node29 = sensors_covered[5::9]
flow_sensors_covered_node1 = sensors_covered[6::9]
flow_sensors_covered_node16 = sensors_covered[7::9]
flow_sensors_covered_node29 = sensors_covered[8::9]

In [ ]:
plt.plot(demand_sensors_covered_node1, demand_errors_mae_node1)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE Error vs. Coverage Demands Node 1")
plt.xlim([100, 0])
plt.show()

plt.plot(demand_sensors_covered_node1, demand_errors_rmse_node1)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Demands Node 1")
plt.xlim([100, 0])
plt.show()

plt.plot(demand_sensors_covered_node1, demand_errors_r2_node1)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 Error vs. Coverage Demands Node 1")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(demand_sensors_covered_node16, demand_errors_mae_node16)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Demands Node 16")
plt.xlim([100, 0])
plt.show()

plt.plot(demand_sensors_covered_node16, demand_errors_rmse_node16)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Demands Node 16")
plt.xlim([100, 0])
plt.show()

plt.plot(demand_sensors_covered_node16, demand_errors_r2_node16)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Demands Node 16")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(demand_sensors_covered_node29, demand_errors_mae_node29)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Demands Node 29")
plt.xlim([100, 0])
plt.show()

plt.plot(demand_sensors_covered_node29, demand_errors_rmse_node29)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Demands Node 29")
plt.xlim([100, 0])
plt.show()

plt.plot(demand_sensors_covered_node29, demand_errors_r2_node29)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Demands Node 29")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(pressure_sensors_covered_node1, pressure_errors_mae_node1)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Pressure Node 1")
plt.xlim([100, 0])
plt.show()

plt.plot(pressure_sensors_covered_node1, pressure_errors_rmse_node1)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Pressure Node 1")
plt.xlim([100, 0])
plt.show()

plt.plot(pressure_sensors_covered_node1, pressure_errors_r2_node1)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Pressure Node 1")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(pressure_sensors_covered_node16, pressure_errors_mae_node16)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Pressure Node 16")
plt.xlim([100, 0])
plt.show()

plt.plot(pressure_sensors_covered_node16, pressure_errors_rmse_node16)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Pressure Node 16")
plt.xlim([100, 0])
plt.show()

plt.plot(pressure_sensors_covered_node16, pressure_errors_r2_node16)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Pressure Node 16")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(pressure_sensors_covered_node29, pressure_errors_mae_node29)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Pressure Node 29")
plt.xlim([100, 0])
plt.show()

plt.plot(pressure_sensors_covered_node29, pressure_errors_rmse_node29)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Pressure Node 29")
plt.xlim([100, 0])
plt.show()

plt.plot(pressure_sensors_covered_node29, pressure_errors_r2_node29)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Pressure Node 29")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(flow_sensors_covered_node1, flow_errors_mae_node1)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Flows Node 1")
plt.xlim([100, 0])
plt.show()

plt.plot(flow_sensors_covered_node1, flow_errors_rmse_node1)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Flows Node 1")
plt.xlim([100, 0])
plt.show()

plt.plot(flow_sensors_covered_node1, flow_errors_r2_node1)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Flows Node 1")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(flow_sensors_covered_node16, flow_errors_mae_node16)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Flows Node 16")
plt.xlim([100, 0])
plt.show()

plt.plot(flow_sensors_covered_node16, flow_errors_rmse_node16)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Flows Node 16")
plt.xlim([100, 0])
plt.show()

plt.plot(flow_sensors_covered_node16, flow_errors_r2_node16)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Flows Node 16")
plt.xlim([100, 0])
plt.show()

In [ ]:
plt.plot(flow_sensors_covered_node29, flow_errors_mae_node29)
plt.ylabel("Mean Absolute Error")
plt.xlabel("Sensor Cover %")
plt.title("MAE vs. Coverage Flows Node 29")
plt.xlim([100, 0])
plt.show()

plt.plot(flow_sensors_covered_node29, flow_errors_rmse_node29)
plt.ylabel("RMSE")
plt.xlabel("Sensor Cover %")
plt.title("RMSE vs. Coverage Flows Node 29")
plt.xlim([100, 0])
plt.show()

plt.plot(flow_sensors_covered_node29, flow_errors_r2_node29)
plt.ylabel("R2")
plt.xlabel("Sensor Cover %")
plt.title("R2 vs. Coverage Flows Node 29")
plt.xlim([100, 0])
plt.show()